**Setup a table**

In [10]:
DROP TABLE IF EXISTS [dbo].[testcustomertable]
GO

CREATE TABLE [dbo].[testcustomertable](
	[customerid] [bigint] NOT NULL,
	[customercode] [varchar](10) NOT NULL,
    [salespersoncode] [varchar](10) NOT NULL,
    [companyname] varchar(128) NOT NULL,
	[title] [varchar](8) NULL default('titlex'),
	[firstname] [nvarchar](64) NULL default('testfirstname'),
	[LastName] [nvarchar](64) NULL default('testlastname')
) 

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.094

**Insert a row with a unique combination** 

Insert #2 will contain companyname with three trailing spaces.

In [11]:
-- #1
INSERT INTO [dbo].[testcustomertable](
    customerid,
    customercode,
    salespersoncode,
    companyname
) VALUES (
1,
'custcode1',
'salescode1',
'companytest'
);

-- #2
INSERT INTO [dbo].[testcustomertable](
    customerid,
    customercode,
    salespersoncode,
    companyname
) VALUES (
    1,
    'custcode1',
    'salescode1',
    'companytest   '
);

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.053

**Review data**

LEN versus DATALENGTH - [LEN (Transact-SQL) - SQL Server | Microsoft Docs](https://docs.microsoft.com/en-us/sql/t-sql/functions/len-transact-sql?view=sql-server-ver15#remarks) - _LEN excludes trailing spaces. If that is a problem, consider using the DATALENGTH (Transact-SQL) function which does not trim the string. If processing a unicode string, DATALENGTH will return a number that may not be equal to the number of characters._

In [4]:
SELECT 
    customerid,
    customercode,
    salespersoncode,
    companyname, 
    LEN([companyname]) AS [companyname_len],
    DATALENGTH([companyname]) AS [companyname_length] 
FROM [dbo].[testcustomertable];

(2 rows affected)

Total execution time: 00:00:00.098

customerid,customercode,salespersoncode,companyname,companyname_len,companyname_length
1,custcode1,salescode1,companytest,11,11
1,custcode1,salescode1,companytest,11,14


**Compare**

[INF: How SQL Server Compares Strings with Trailing Spaces](https://support.microsoft.com/nl-nl/topic/inf-how-sql-server-compares-strings-with-trailing-spaces-b62b1a2d-27d3-4260-216d-a605719003b0): _SQL Server follows the ANSI/ISO SQL-92 specification (Section 8.2, \<Comparison Predicate\>, General rules #3) on how to compare strings with spaces. The ANSI standard requires padding for the character strings used in comparisons so that their lengths match before comparing them. The padding directly affects the semantics of WHERE and HAVING clause predicates and other Transact-SQL string comparisons. For example, Transact-SQL considers the strings 'abc' and 'abc ' to be equivalent for most comparison operations._

In [9]:
-- Both records will be returned
SELECT *, DATALENGTH(companyname) AS companyname_datalength
FROM [dbo].[testcustomertable]
WHERE [companyname] = 'companytest   ';

(2 rows affected)

Total execution time: 00:00:00.131

customerid,customercode,salespersoncode,companyname,title,firstname,LastName,companyname_datalength
1,custcode1,salescode1,companytest,titlex,testfirstname,testlastname,11
1,custcode1,salescode1,companytest,titlex,testfirstname,testlastname,14


In [11]:
-- Use datalength to limit down the where-clause
SELECT *, DATALENGTH(companyname) AS companyname_datalength
FROM [dbo].[testcustomertable]
WHERE [companyname] = 'companytest   '
AND DATALENGTH([companyname]) = DATALENGTH('companytest   ');

(1 row affected)

Total execution time: 00:00:00.050

customerid,customercode,salespersoncode,companyname,title,firstname,LastName,companyname_datalength
1,custcode1,salescode1,companytest,titlex,testfirstname,testlastname,14


In [8]:
-- Add a character in the comparison
SELECT *, DATALENGTH(companyname) AS companyname_datalength
FROM [dbo].[testcustomertable]
WHERE [companyname] + 'x' = 'companytest   ' + 'x'
AND DATALENGTH([companyname]) = DATALENGTH('companytest   ');

(1 row affected)

Total execution time: 00:00:00.106

customerid,customercode,salespersoncode,companyname,title,firstname,LastName,companyname_datalength
1,custcode1,salescode1,companytest,titlex,testfirstname,testlastname,14


In [12]:
SELECT 
    [companyname],
    CASE WHEN [companyname] = 'companytest' THEN 'MATCH' ELSE 'NO MATCH' END,
    CASE WHEN [companyname] + 'x' = 'companytestx' THEN 'MATCH' ELSE 'NO MATCH' END
FROM [dbo].[testcustomertable];

(2 rows affected)

Total execution time: 00:00:00.127

companyname,(No column name),(No column name)
companytest,MATCH,MATCH
companytest,MATCH,NO MATCH


## **Create Unique index on <span style="color: rgb(33, 33, 33); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">customerid, c</span><span style="color: rgb(33, 33, 33); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">ustomercode, </span> <span style="color: rgb(33, 33, 33); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">salespersoncode, </span> <span style="color: rgb(33, 33, 33); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">companyname</span>**

[INF: Insertion of Strings Containing Trailing Spaces:](https://www.betaarchive.com/wiki/index.php/Microsoft_KB_Archive/231830) <span style="color: rgb(33, 37, 41); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Roboto, &quot;Helvetica Neue&quot;, Arial, &quot;Noto Sans&quot;, &quot;Liberation Sans&quot;, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;, &quot;Noto Color Emoji&quot;; font-size: 16px; background-color: rgb(255, 255, 255);"><i>An insertion into a column whose values must be unique will fail if you supply a value that is differentiated from existing values by trailing spaces only. The following strings will all be considered equivalent by a unique constraint, primary key, or unique index. Likewise, if you have an existing table with the data below and try to add a unique restriction, it will fail because the values are considered identical.</i></span>

In [15]:
CREATE UNIQUE NONCLUSTERED INDEX [IDX_UNIQUECOMBINATION]
    ON [dbo].[testcustomertable]([customerid] ASC, [customercode] ASC, [salespersoncode] ASC, [companyname] ASC);


: Msg 1505, Level 16, State 1, Line 1
The CREATE UNIQUE INDEX statement terminated because a duplicate key was found for the object name 'dbo.testcustomertable' and the index name 'IDX_UNIQUECOMBINATION'. The duplicate key value is (1, custcode1, salescode1, companytest   ).

The statement has been terminated.

Total execution time: 00:00:00.059

**Alternative 1 - Add computed column with trailing character**

In [13]:
-- Alternative one - Add trailing character
ALTER TABLE [dbo].[testcustomertable]
ADD [companyname_trailing_character] AS CAST(companyname + 'x' AS VARCHAR(129));

Commands completed successfully.

Total execution time: 00:00:00.079

In [ ]:
CREATE UNIQUE NONCLUSTERED INDEX [IDX_UNIQUECOMBINATION_1]
    ON [dbo].[testcustomertable]([customerid] ASC, [customercode] ASC, [salespersoncode] ASC, [companyname_trailing_character] ASC);

**Alternative 2 - Add computed column with datalength of companyname**

In [16]:
-- Alternative one - Add trailing character
ALTER TABLE [dbo].[testcustomertable]
ADD [companyname_length] AS DATALENGTH(companyname);

: Msg 2705, Level 16, State 5, Line 2
Column names in each table must be unique. Column name 'companyname_length' in table 'dbo.testcustomertable' is specified more than once.

Total execution time: 00:00:00.044

In [20]:
DROP INDEX IF EXISTS [IDX_UNIQUECOMBINATION_2] ON [dbo].[testcustomertable]

CREATE UNIQUE NONCLUSTERED INDEX [IDX_UNIQUECOMBINATION_2]
    ON [dbo].[testcustomertable]([customerid] ASC, [customercode] ASC, [salespersoncode] ASC, [companyname] ASC, [companyname_length] ASC);

Commands completed successfully.

Total execution time: 00:00:00.118

In [21]:
SELECT *
FROM [dbo].[testcustomertable];

(2 rows affected)

Total execution time: 00:00:00.047

customerid,customercode,salespersoncode,companyname,title,firstname,LastName,companyname_trailing_character,companyname_length
1,custcode1,salescode1,companytest,titlex,testfirstname,testlastname,companytestx,11
1,custcode1,salescode1,companytest,titlex,testfirstname,testlastname,companytest x,14
